# Exploratory Data Analysis

In [1]:
# import neccessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as dt 

c:\Users\TEJAS\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# load the dataset
customers_df = pd.read_csv('../datasets/Customers.csv')
products_df = pd.read_csv('../datasets/Products.csv')
transactions_df = pd.read_csv('../datasets/Transactions.csv')

## 1. Data Overview and Descriptive Statistics

In [3]:
# customers dataset
customers_df.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [4]:
# products dataset
products_df.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [5]:
# transactions dataset
transactions_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 7:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [6]:
# high-level overview of customers dataset
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


##### We observe that the column SignupDate is being read as an object data type

In [7]:
# convert SignupDate to datetime format
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
customers_df['SignupDate'].head()


0   2022-07-10
1   2022-02-13
2   2024-03-07
3   2022-10-09
4   2022-08-15
Name: SignupDate, dtype: datetime64[ns]

In [8]:
# high-level overview of products dataset
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB


In [9]:
# high-level overview of transactions dataset
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price            1000 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB


##### We observe that the column TransactionDate is being read as an object data type

In [10]:
# convert TransactionDate to datetime format
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])
transactions_df['TransactionDate'].head()

0   2024-08-25 12:38:23
1   2024-05-27 22:23:54
2   2024-04-25 07:38:55
3   2024-03-26 22:55:37
4   2024-03-21 15:10:10
Name: TransactionDate, dtype: datetime64[ns]

In [11]:
# customers dataset info
print('Shape of customers dataset:', customers_df.shape)

print('\nData-types of customers dataset:\n', customers_df.dtypes)

print('\nDiscriptive statistics of customers dataset:\n', customers_df.describe())

Shape of customers dataset: (200, 4)

Data-types of customers dataset:
 CustomerID              object
CustomerName            object
Region                  object
SignupDate      datetime64[ns]
dtype: object

Discriptive statistics of customers dataset:
                 SignupDate
count                  200
mean   2023-07-19 08:31:12
min    2022-01-22 00:00:00
25%    2022-09-26 12:00:00
50%    2023-08-31 12:00:00
75%    2024-04-12 12:00:00
max    2024-12-28 00:00:00


In [12]:
# products dataset info
print('Shape of products dataset:', products_df.shape)
print('\nData-types of products dataset:\n', products_df.dtypes)
print('\nDiscriptive statistics of products dataset:\n', products_df.describe())

Shape of products dataset: (100, 4)

Data-types of products dataset:
 ProductID       object
ProductName     object
Category        object
Price          float64
dtype: object

Discriptive statistics of products dataset:
             Price
count  100.000000
mean   267.551700
std    143.219383
min     16.080000
25%    147.767500
50%    292.875000
75%    397.090000
max    497.760000


In [13]:
# transactions dataset info
print('Shape of transactions dataset:', transactions_df.shape)
print('\nData-types of transactions dataset:\n', transactions_df.dtypes)
print('\nDiscriptive statistics of transactions dataset:\n', transactions_df.describe())

Shape of transactions dataset: (1000, 7)

Data-types of transactions dataset:
 TransactionID              object
CustomerID                 object
ProductID                  object
TransactionDate    datetime64[ns]
Quantity                    int64
TotalValue                float64
Price                     float64
dtype: object

Discriptive statistics of transactions dataset:
                      TransactionDate     Quantity   TotalValue       Price
count                           1000  1000.000000  1000.000000  1000.00000
mean   2024-06-23 15:33:02.768999936     2.537000   689.995560   272.55407
min              2023-12-30 15:29:12     1.000000    16.080000    16.08000
25%       2024-03-25 22:05:34.500000     2.000000   295.295000   147.95000
50%       2024-06-26 17:21:52.500000     3.000000   588.880000   299.93000
75%              2024-09-19 14:19:57     4.000000  1011.660000   404.40000
max              2024-12-28 11:00:00     4.000000  1991.040000   497.76000
std                

## 2. Data Preprocessing

In [14]:
# Check for missing values
print("\nMissing values in Customers DataFrame:")  
print(customers_df.isnull().sum())  
print("\nMissing values in Products DataFrame:")  
print(products_df.isnull().sum())  
print("\nMissing values in Transactions DataFrame:")  
print(transactions_df.isnull().sum()) 


Missing values in Customers DataFrame:
CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64

Missing values in Products DataFrame:
ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64

Missing values in Transactions DataFrame:
TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


#### No missing values found

In [15]:
# Drop duplicates if any
customers_df.drop_duplicates(inplace=True)
products_df.drop_duplicates(inplace=True)
transactions_df.drop_duplicates(inplace=True)

In [16]:
#    
reference_date = pd.Timestamp(dt.now().date())
reference_date = transactions_df['TransactionDate'].max() + pd.Timedelta(days=1)

In [17]:
# merge all three dataframes
df = transactions_df.merge(customers_df, on='CustomerID', how='left').merge(products_df, on='ProductID', how='left')
df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [18]:
# drop unnecessary columns
df.drop(['Price_y'], axis=1, inplace=True)

# rename columns
df.rename(columns={'Price_x': 'Price'}, inplace=True)
df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,CustomerName,Region,SignupDate,ProductName,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics


In [19]:
# Recency, Frequency, Monetary Value
rfm = df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (reference_date - x.max()).days, # Find most recent transaction date - Recency
    'TransactionID': 'count',  # Count of transactions - Frequency
    'Price': 'sum' # Sum of total spend - Monetary Value
})

In [20]:
# rename columns
rfm = rfm.rename(columns={'TransactionDate': 'Recency',
                          'TransactionID': 'Frequency',
                          'Price': 'MonetaryValue'}, inplace=False)

rfm.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
C0001,56,5,1391.67
C0002,26,4,835.68
C0003,126,4,782.83
C0004,5,8,1925.09
C0005,55,3,874.81


In [21]:
# define quantiles
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])

# assign RFM scores
def RScore(x, p, d):
    if p == 'Recency': # lower the recency, higher the score
        if x <= d[p][0.25]:
            return 4
        elif x <= d[p][0.50]:
            return 3
        elif x <= d[p][0.75]:
            return 2
        else:
            return 1
    else: # higher the frequency/monetary value, higher the score
        if x <= d[p][0.25]:
            return 1
        elif x <= d[p][0.50]:
            return 2
        elif x <= d[p][0.75]:
            return 3
        else:
            return 4
        
rfm['R'] = rfm['Recency'].apply(RScore, args=('Recency', quantiles))
rfm['F'] = rfm['Frequency'].apply(RScore, args=('Frequency', quantiles))
rfm['M'] = rfm['MonetaryValue'].apply(RScore, args=('MonetaryValue', quantiles)) 

rfm.head()


,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
C0001,56,5,1391.67,3,2,3
C0002,26,4,835.68,3,2,1
C0003,126,4,782.83,1,2,1
C0004,5,8,1925.09,4,4,4
C0005,55,3,874.81,3,1,1


In [22]:
# calculate RFM score
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score
CustomerID,,,,,,,
C0001,56,5,1391.67,3,2,3,8
C0002,26,4,835.68,3,2,1,6
C0003,126,4,782.83,1,2,1,4
C0004,5,8,1925.09,4,4,4,12
C0005,55,3,874.81,3,1,1,5


In [23]:
# segment customers based on RFM score
segment_labels = ['Low-value', 'Mid-value', 'High-value']

def assign_segment(x):
    if x <= 5:
        return segment_labels[0]
    elif x <= 9:
        return segment_labels[1]
    else:
        return segment_labels[2]
    
rfm['RFM_segment_label'] = rfm['RFM_Score'].apply(assign_segment)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,RFM_segment_label
CustomerID,,,,,,,,
C0001,56,5,1391.67,3,2,3,8,Mid-value
C0002,26,4,835.68,3,2,1,6,Mid-value
C0003,126,4,782.83,1,2,1,4,Low-value
C0004,5,8,1925.09,4,4,4,12,High-value
C0005,55,3,874.81,3,1,1,5,Low-value


## 3. Visualizations

### Trends over time

In [24]:
# 1. monthly transaction counts
monthly_transactions_count = transactions_df.resample('M', on='TransactionDate').size()
monthly_transactions_count

C:\Users\TEJAS\AppData\Local\Temp\ipykernel_5296\1880014757.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_transactions_count = transactions_df.resample('M', on='TransactionDate').size()


TransactionDate
2023-12-31      4
2024-01-31    107
2024-02-29     77
2024-03-31     80
2024-04-30     86
2024-05-31     86
2024-06-30     69
2024-07-31     96
2024-08-31     94
2024-09-30     96
2024-10-31     70
2024-11-30     57
2024-12-31     78
Freq: ME, dtype: int64

In [25]:
# convert index to months
monthly_transactions_count.index = monthly_transactions_count.index.strftime('%b %Y')

In [26]:
# plot bar graph of monthly transaction counts
fig = px.bar(x=monthly_transactions_count.index,
             y=monthly_transactions_count.values,
             labels={'x': 'Months', 'y': 'Transaction Counts'},
             title='Monthly Transaction Counts')
fig.show()

In [27]:
# 2. Monthly total transaction values
monthly_values = transactions_df.resample('ME', on='TransactionDate')['TotalValue'].sum() 
# monthly_values.index = monthly_values.index.strftime('%b %Y') 
# plt.figure(figsize=(12, 6))  
# monthly_values.plot(kind='line', marker='o', color='green')  
# plt.title('Monthly Total Transaction Values')  
# plt.xlabel('Month')  
# plt.ylabel('Total Value')  
# plt.xticks(rotation=45)  
# plt.show()  

fig = px.line(x=monthly_values.index,
              y=monthly_values.values,
              labels={'x': 'Months', 'y': 'Total Value'},
              title='Monthly Total Transaction Values')

fig.show()

c:\Users\TEJAS\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Product Analysis

In [28]:
# 1. Top selling products
top_products = transactions_df.groupby('ProductID')['Quantity'].sum().nlargest(10).reset_index()  # Convert to DataFrame  
top_products = top_products.merge(products_df[['ProductID', 'ProductName']], on='ProductID', how='left')  

In [29]:
fig = px.bar(x=top_products['ProductName'],
             y=top_products['Quantity'],
             labels={'x': 'Products', 'y': 'Quantity'},
             title='Top 10 Selling Products',
             color=top_products['Quantity'],
             color_continuous_scale='Viridis')

fig.show()

## Customer Engagement

In [30]:
# rfm segment counts
segment_counts = rfm['RFM_segment_label'].value_counts().reset_index()
segment_counts.columns = ['RFM_segment_label', 'Counts']

In [31]:
# create a bar chart
fig = px.bar(segment_counts,
             x='RFM_segment_label',
             y='Counts',
             title='Customer distribution by RFM segments',
             labels = {'RFM_segment_label': 'RFM Segment', 'Counts': 'Number of Customers'},
             color='RFM_segment_label',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

c:\Users\TEJAS\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [32]:
# RFM customer segments
rfm['RFM_customer_segment'] = ''

rfm.loc[rfm['RFM_Score'] > 9, 'RFM_customer_segment'] = 'Champion'
rfm.loc[(rfm['RFM_Score'] > 6) & (rfm['RFM_Score'] <= 9), 'RFM_customer_segment'] = 'Promising'
rfm.loc[(rfm['RFM_Score'] > 5) & (rfm['RFM_Score'] <= 6), 'RFM_customer_segment'] = 'At Risk'
rfm.loc[(rfm['RFM_Score'] > 4) & (rfm['RFM_Score'] <= 5), 'RFM_customer_segment'] = 'Can\'t Lose'
rfm.loc[(rfm['RFM_Score'] <= 4), 'RFM_customer_segment'] = 'Lost'

segment_counts = rfm['RFM_customer_segment'].value_counts().reset_index()

In [33]:
segment_product_counts = rfm.groupby(['RFM_segment_label', 'RFM_customer_segment']).size().reset_index(name='Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

In [34]:
# create a tree map
fig = px.treemap(segment_product_counts,
                 path = ['RFM_segment_label', 'RFM_customer_segment'],
                 values = 'Count',
                 color = 'RFM_segment_label',
                 color_discrete_sequence=px.colors.qualitative.Pastel,
                 title='RFM Customer Segments by value')

# display the treemap
fig.show()

In [35]:
# Heatmap of Champions segment
correlation_matrix = rfm[rfm['RFM_customer_segment'] == 'Champion'][['R', 'F', 'M']].corr()

fig = go.Figure(data=go.Heatmap(
                     z=correlation_matrix.values,
                     x=correlation_matrix.columns,
                     y=correlation_matrix.columns,
                     colorscale='Viridis',
                     colorbar=dict(title='Correlation Coefficient')))

fig.update_layout(title='Correlation Matrix of RFM values within Champion Segment')

fig.show()


In [36]:
# compare the RFM values for each segment
segment_scores = rfm.groupby('RFM_customer_segment')[['R', 'F', 'M']].mean().reset_index()

fig = go.Figure()

# add trace for Recency
fig.add_trace(go.Bar(x=segment_scores['RFM_customer_segment'],
                     y=segment_scores['R'],
                     name='Recency'))

# add trace for Frequency
fig.add_trace(go.Bar(x=segment_scores['RFM_customer_segment'],
                     y=segment_scores['F'],
                     name='Frequency'))

# add trace for Monetary Value
fig.add_trace(go.Bar(x=segment_scores['RFM_customer_segment'],
                     y=segment_scores['M'],
                     name='Monetary Value'))

fig.update_layout(title='Comparison of Segments based on Recency, Frequency and Monetary values',
                  xaxis_title='RFM Segment',
                  yaxis_title='Score',
                  barmode = 'group',
                  showlegend=True)

fig.show()

In [37]:
# Save the RFM data to a csv file
rfm.to_csv('../datasets/RFM_model.csv', index=False)
print("RFM model saved as 'RFM_model.csv'")

RFM model saved as 'RFM_model.csv'


In [38]:
# Save DataFrame to CSV  
df.to_csv('../datasets/merged_data.csv', index=False)  
print("DataFrame saved as 'merged_data.csv'")  

DataFrame saved as 'merged_data.csv'


## 4. Insights